# Breadth First Search
###### (https://doisinkidney.com/posts/2018-03-17-rose-trees-breadth-first.html)
Breadth first searches are traditionally implemented using a data structure resembling a queue. In Haskell we can do this using `Data.Sequence` or two lists. However, both of these are more than we really need.

In [14]:
data Tree a = Node a (Forest a)
type Forest a = [Tree a]

exampleTree :: Tree Char
exampleTree = Node '1' [t1, t2, t3] where
  t1 = Node 'q' [Node 'a' [], Node 's' []]
  t2 = Node 'w' [Node 'd' [Node 'z' []]]
  t3 = Node 'e' []

breadthFirst :: Tree a -> [a]
breadthFirst tr = go (singleton tr) where
  go q = case pop q of
    Nothing -> []
    Just (Node x xs, qs) -> x : go (qs `append` xs)
    
-- singleton, pop, and append are part of some queue API
-- could be implemented very naively as
type Queue a = [a]
singleton :: a -> Queue a
singleton = (:[])

pop :: Queue a -> Maybe (a, Queue a)
pop [] = Nothing
pop (x:xs) = Just (x, xs)

append = (++)

Instead of `pop`, The `go` function can be expressed as a fold.
The `append` does not need to stick the items onto the queue immediately, it can batch them, first appending to a structure that is efficient for appends, then converting to a structure that is efficient for folds.

In [16]:
:ext ScopedTypeVariables

breadthFirst :: Forest a -> [a]
breadthFirst ts = b [ts] where --foldr f b ts [] where
  f :: Tree a -> ([Forest a] -> [a]) -> [Forest a] -> [a]
  f (Node x xs) fw bw = x : fw (xs : bw) -- will be consumed in the right order due to left fold
  
  b :: [Forest a] -> [a]
  b [] = []
  b qs = foldl (foldr f) b qs ([] :: [Forest a]) -- foldr f q1 b $ foldr f q2 b $ foldr f q3 b $ []
  
breadthFirst [exampleTree]

"1qweasdz"

The `b` function does a right fold over a list of `Forest a`s, turning it into a list of `a`s in breadth first order. So it goes left to right, running `foldr f b q` on each forest. The result of this `foldl` is a function that takes a list of Forests which will be the argument to the result of the `foldr` that gets applied to the last Forest in the input list. This list of forests is just an empty list, so it would look like `foldr f b q []`.
This `foldr` applies `f x b` to each tree in the forest. This produces a function that takes a `[Forest a]` and chains these functions together.